In [2]:
# -------------------------------------------------------------------------------------
# Libraries
import os
import numpy as np
import pandas as pd
import geopandas as gdp
from matplotlib import pyplot as plt

In [19]:
# -------------------------------------------------------------------------------------
# Inputs
sCsvNameIn = 'input/SequiaVIDECI_2002_2019.csv' 

sTimeAgg = 'M' # Y = yearly; M = monthly
sSpaceAgg = 'Mu' # D = Departments; Mu = Municipalities; MR = MacroRegions

sDepNameIn = 'input/shp/BOL_departamentos.shp'
sMunNameIn = 'input/shp/BOL_municipios.shp'
sMRegNameIn = 'input/shp/BOL_macroregiones.shp'
sFldOut = 'output'

# Plot (select date)


In [43]:
# Import VIDECI Impact dataset (csv format, revision)
df = pd.read_csv(sCsvNameIn,delimiter=",", encoding='utf-8')

# Import Departments Shapefile
gfd = gdp.read_file(sDepNameIn)

# Import Municipalities Shapefile
gfm = gdp.read_file(sMunNameIn)

# Import MacroRegion Shapefile
gfr = gdp.read_file(sMRegNameIn)

# Convert dates to datetime
df['event_date'] = pd.to_datetime(df['event_date'], format="%d/%m/%Y")
# Set time index
df = df.set_index(["event_date"])


# Get people affected per year - total
df_m_tot = df['p_afect'].resample(sTimeAgg, convention='end').sum()
dfm = df_m_tot.to_frame()
dDates = dfm.index.values
final_col_names = ['Total_BO']

if sSpaceAgg == 'D':  
    # Get people affected per year for each department
    for depart in df.Dep.unique():
        
        df_sel = df[df['Dep']==depart]
        df_m = df_sel['p_afect'].resample(sTimeAgg, convention='end').sum()
        dfm = pd.concat([dfm, df_m], axis=1, sort=False)
        final_col_names.append(depart)
        del df_m     
        
elif sSpaceAgg == 'Mu':
    # Get people affected per year for each municipality # With code..safer
    for muncode in df.cod_mun.unique():
        
        df_sel = df[df['cod_mun']==muncode]
        df_m = df_sel['p_afect'].resample(sTimeAgg, convention='end').sum()
        dfm = pd.concat([dfm, df_m], axis=1, sort=False)
        final_col_names.append(muncode)
        del df_m
        
elif sSpaceAgg == 'MR':
    # Get people affected per year for each macroregion
    for macroreg in df.MacroReg.unique():
        
        df_sel = df[df['MacroReg']==macroreg]
        df_m = df_sel['p_afect'].resample(sTimeAgg, convention='end').sum()
        dfm = pd.concat([dfm, df_m], axis=1, sort=False)
        final_col_names.append(macroreg)
        del df_m   
else:
    print('Chose correct Space Aggregation: Y or Mu')
    
dfm.columns = final_col_names

if sTimeAgg == 'Y':
    dfm['Year']=pd.to_datetime(dfm.index.values).year
    dfm = dfm.set_index(['Year'])
    if sSpaceAgg == 'D' or sSpaceAgg == 'MR':
        dfm.plot.bar()
        sGraphName = 'impactos_BOLIVIA_2002_2019_' + sTimeAgg + '_' + sSpaceAgg + '.png'
        sSaveName = os.path.join(sFldOut,sGraphName)
        plt.savefig(sSaveName)
elif sTimeAgg == 'M':
    dfm['Yr-Month']=pd.to_datetime(dfm.index.values).to_period('M')
    dfm = dfm.set_index(['Yr-Month']) 
else:
    print('Chose correct Time Aggregation: Y or M')
    
# Convert to Shapefile
# transpose dataframe and remove first line (TOTAL_BO)
dfmt = dfm.transpose()
dfmts = dfmt.drop('Total_BO')
dfmtsbkp = dfmts
dfmtsbkp = dfmtsbkp.fillna(0)

dfmts['SUM'] = np.nan
dfmts['MEAN'] = np.nan
dfmts = dfmts.fillna(0)
#dfmts = dfmts.astype(int)
if sSpaceAgg == 'D':
    for depart in df.Dep.unique():
        dfmts.loc[depart,'SUM'] = np.sum(dfmtsbkp.loc[depart])
        dfmts.loc[depart,'MEAN'] = np.mean(dfmtsbkp.loc[depart])
        
    dfmts['Dep'] = dfmts.index
    dfmts.columns = dfmts.columns.map(str)
    gfs_merge = gfd.merge(dfmts, on='Dep')
elif sSpaceAgg == 'Mu':
    dfmts.columns = dfmts.columns.map(str)
    dfmts['COD1_SECC'] = dfmts.index
    dfmts.COD1_SECC = dfmts.COD1_SECC.astype(float)
    gfm.COD1_SECC = gfm.COD1_SECC.astype(float)
    gfs_merge = gfm.merge(dfmts, on='COD1_SECC')
elif sSpaceAgg == 'MR':
    # Get people affected per year for each macroregion
    for macroreg in df.MacroReg.unique():
        dfmts.loc[macroreg,'SUM'] = np.sum(dfmtsbkp.loc[macroreg])
        dfmts.loc[macroreg,'MEAN'] = np.mean(dfmtsbkp.loc[macroreg])
        
    dfmts['MacroReg'] = dfmts.index
    dfmts.columns = dfmts.columns.map(str)
    gfs_merge = gfr.merge(dfmts, on='MacroReg')
        
else:
    print('Chose correct Space Aggregation: D or Mu')

sShpNameOut = 'impactos_BOLIVIA_2002_2019_' + sTimeAgg + '_' + sSpaceAgg + '.shp'
sSaveName = os.path.join(sFldOut,'shp',sShpNameOut)
gfs_merge.to_file(sSaveName)    

DriverIOError: Failed to create file output/shp/impactos_BOLIVIA_2002_2019_M_Mu.shp: No such file or directory